In [1]:
# 1

import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup 
from keras import layers
from keras import losses

In [2]:
# 2

import pandas as pd

train_df = pd.read_csv('/Users/jacobjohnson/data_sets/congressional_tweet_training_data.csv', names=['favorite_count', 'full_text', 'hashtags', 'retweet_count', 'year', 'party_id'], 
                    # dtype={'favorite_count': int, 'full_text': str, 'hashtags': str, 'retweet_count': int, 'year': int, 'party_id': str}, 
                    skipinitialspace=True, skiprows=1, sep=',')

test_df = pd.read_csv('/Users/jacobjohnson/data_sets/congressional_tweet_test_data.csv', names=['id', 'favorite_count_test', 'full_text_test', 'hashtags_test', 'retweet_count_test', 'year', 'party'], 
                    # dtype={'id': int, 'favorite_count': int, 'full_text': str, 'hashtags': str, 'retweet_count': int, 'year': int, 'party': str}, 
                    skipinitialspace=True, skiprows=1, sep=',')

train_df.pop('year')
test_df.pop('year')

# test_df.head()
train_df.head()

# target variable
# party = train_df.pop('party_id')



,favorite_count,full_text,hashtags,retweet_count,party_id
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,R
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,R
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,R
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,R
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,D


In [3]:
# 3

train, val, test = np.split(train_df.sample(frac=1), [int(0.8*len(train_df)), int(0.9*len(train_df))])

print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

474242 training examples
59280 validation examples
59281 test examples


In [4]:
# 4

def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  df = dataframe.copy()
  names = df.pop('party_id')
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df), names))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [10]:
# 5

# batch_size = 5
# train_ds = df_to_dataset(train, batch_size=batch_size)

batch_size = 256
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

/var/folders/nf/brcqg8zx7xg31hjn_xbd2jc80000gp/T/ipykernel_32165/2315459290.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
/var/folders/nf/brcqg8zx7xg31hjn_xbd2jc80000gp/T/ipykernel_32165/2315459290.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
/var/folders/nf/brcqg8zx7xg31hjn_xbd2jc80000gp/T/ipykernel_32165/2315459290.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()

In [11]:
# 6

[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of hashtags:', train_features['hashtags'])
print('A batch of targets:', label_batch )

Every feature: ['favorite_count', 'full_text', 'hashtags', 'retweet_count', 'party_id']
A batch of hashtags: tf.Tensor(
[[b'va01']
 [b'TradeSummit2018']
 [b'TN01 iloveTNteachers NationalTeacherAppreciationDay']
 [b'ClimateChange']
 [b'BoomerSooner']
 [b'Bronx']
 [b'MadeInAmerica']
 [b'NorthDakota Bakken']
 [b'LA04 shopsmall SmallBusinessWeek']
 [b'MN03']
 [b'NetNeutrality OneMoreVote SaveTheInternet']
 [b'TaxBill']
 [b'SmallBusiness']
 [b'opioids Missouri']
 [b'PatFinucane UnifinishedBusiness']
 [b'Willowbrook CA44']
 [b'Haiyan']
 [b'EidMubarak']
 [b'SOTU Ag SpaceForce AL03']
 [b'CCE']
 [b'Texas America FridayFeelting tcot']
 [b'MobileOffice']
 [b'POTUS']
 [b'cosgrads2019']
 [b'ObamaCare ObamaCare KlineTTH']
 [b'PA06']
 [b'NY13 ForThePeople']
 [b'Obamacare']
 [b'ATF']
 [b'COVID19']
 [b'WA']
 [b'SWFL']
 [b'ProtectingOurInfantsAct']
 [b'FamiliesBelongTogether']
 [b'CIR']
 [b'BetterOffNow']
 [b'TrumpSwamp ForThePeople']
 [b'nationalprayerbreakfast']
 [b'AEW15chat']
 [b'FundForAmericasKids

In [7]:
# 7

def get_normalization_layer(name, dataset):
  normalizer = layers.Normalization(axis=None)
  feature_ds = dataset.map(lambda x, y: x[name])
  normalizer.adapt(feature_ds)
  return normalizer

def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  if dtype == 'string':
    # DOES THIS REQUIRE ALL COLUMNS HAVE DTYPES??
    index = layers.StringLookup(max_tokens=max_tokens)
  else:
    index = layers.IntegerLookup(max_tokens=max_tokens)
  feature_ds = dataset.map(lambda x, y: x[name])
  index.adapt(feature_ds)
  encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
  return lambda feature: encoder(index(feature))

In [31]:
# 8

# Numerical features.

all_inputs = []
encoded_features = []

for header in ['favorite_count', 'retweet_count']:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

In [32]:
# 9

categorical_cols = ['full_text', 'hashtags', 'party_id']

for header in categorical_cols:
  categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
  encoding_layer = get_category_encoding_layer(name=header,
                                               dataset=train_ds,
                                               dtype='string',
                                               max_tokens=None)
  encoded_categorical_col = encoding_layer(categorical_col)
  all_inputs.append(categorical_col)
  encoded_features.append(encoded_categorical_col)

In [34]:
# 10

party_id_col = tf.keras.Input(shape=(1,), name='party_id', dtype='string')

encoding_layer = get_category_encoding_layer(name='party_id',
                                             dataset=train_ds,
                                             dtype='string',
                                             max_tokens=None)
encoded_party_id_col = encoding_layer(party_id_col)
all_inputs.append(party_id_col)
encoded_features.append(encoded_party_id_col)

In [35]:
# 11

test_party_id_col = train_features['party_id']
test_party_id_layer = get_category_encoding_layer(name='party_id',
                                             dataset=train_ds,
                                             dtype='string',
                                             max_tokens=None)
test_party_id_layer(test_party_id_col)

<tf.Tensor: shape=(256, 3), dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
     

In [12]:
# 12
 
test_retweet_count_col = train_features['retweet_count']
test_retweet_count_layer = get_category_encoding_layer(name='retweet_count', dataset=train_ds, dtype='int')
test_retweet_count_layer(test_retweet_count_col)

<tf.Tensor: shape=(256, 5018), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [13]:
# 13

test_favorite_count_col = train_features['favorite_count']
test_favorite_count_layer = get_category_encoding_layer(name='favorite_count', dataset=train_ds, dtype='int')
test_favorite_count_layer(test_favorite_count_col)

<tf.Tensor: shape=(256, 7918), dtype=float32, numpy=
array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [8]:
# 14

test_full_text_col = train_features['full_text']
test_full_text_layer = get_category_encoding_layer(name='full_text', dataset=train_ds, dtype='string')
test_full_text_layer(test_full_text_col)

<tf.Tensor: shape=(5, 470945), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [9]:
# 15

test_hashtags_col = train_features['hashtags']
test_hashtags_layer = get_category_encoding_layer(name='hashtags', dataset=train_ds, dtype='string')
test_hashtags_layer(test_hashtags_col)

<tf.Tensor: shape=(5, 158956), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [16]:
# 16

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(all_inputs, output)

In [17]:
# 17

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [24]:
# 18

model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['party_id'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
2022-05-06 10:30:15.593547: W tensorflow/core/framework/op_kernel.cc:1722] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'binary_crossentropy/Cast' defined at (most recent call last):
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/pyth